In [ ]:
from simu_PSF_polar import *
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [5,3]

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using GPU")
else:   
    device = torch.device('cpu')
    print("Using CPU")

In [ ]:
def limit(x, lim, slope, upper=True):
    '''
    if upper:
       return torch.sum(torch.tensor(1/(1+torch.exp(-slope*(x-lim))), requires_grad=True, device=device))
    else:
        return torch.sum(torch.tensor(1/(1+torch.exp(slope*(x-lim))), requires_grad=True, device=device))
    '''
    if upper:
        return torch.sum(torch.exp((x-lim)*slope))
    else:
        return torch.sum(torch.exp(-1*(x-lim)*slope))
        

In [ ]:
N_photons= torch.tensor(3000., device=device)
N=torch.tensor(80, device=device)
l_pixel=torch.tensor(16, device=device)
NA=torch.tensor(1.4, device=device)
mag=torch.tensor(100, device=device)
lambd=torch.tensor(617, device=device)
f_tube=torch.tensor(200, device=device)
MAG=torch.tensor(200/150, device=device)

In [ ]:
xp = torch.tensor([0. for k in range(300)], device=device)
yp = torch.tensor([0. for k in range(300)], device=device)
z = torch.tensor((3*np.random.rand(300)), device=device) #position of dipole in lambda units
d_ = torch.tensor([-1.5 for k in range(300)], device=device) #defocus of dipole in lambda units 4.4
rho = torch.tensor(10.+160.*np.random.rand(300), requires_grad=True, device=device)
eta = torch.tensor([45. for k in range(300)], requires_grad=True, device=device)# 70
delta = torch.tensor([80. for k in range(300)], requires_grad=True, device=device)#50

In [ ]:
x, y, th1, phi, [Ex0, Ex1, Ex2], [Ey0, Ey1, Ey2], r, r_cut, k, f_o = vectorial_BFP_perfect_focus(N, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, device=device)

In [ ]:
second_plane = torch.tensor([0.35, 0, -0.35], device=device)
polar_projections = torch.tensor([0., 45., 0.], device=device)

In [ ]:
u, v, M = compute_M(xp=xp, yp=yp, zp=z, d=d_, x=x, y=y, th1=th1, phi=phi, Ex0=Ex0, Ex1=Ex1, Ex2=Ex2
                    , Ey0=Ey0, Ey1=Ey1, Ey2=Ey2, r=r, r_cut=r_cut, k=k, f_o=f_o, second_plane=second_plane, polar_projections=polar_projections, N=N, l_pixel=l_pixel, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, MAG=MAG,
                   device=device, aberrations=False, defocus_coef=1e-5, spherical_coef=-1.5)

In [ ]:
psf = PSF(rho=rho, eta=eta, delta=delta, M=M, N_photons=N_photons)

In [ ]:
nn = 12
print('z = ', z[nn].cpu().detach().numpy())

vmax=torch.max(psf[nn]).cpu().detach().numpy()
vmin=torch.min(psf[nn]).cpu().detach().numpy()

plt.rcParams['figure.figsize'] = [15,9]
fig, ax = plt.subplots(2,3)
mesh1 = ax[0,0].pcolormesh(u.cpu(), v.cpu(), psf[nn,0,0].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh2 = ax[1,0].pcolormesh(u.cpu(), v.cpu(), psf[nn,0,1].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh3 = ax[0,1].pcolormesh(u.cpu(), v.cpu(), psf[nn,1,0].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh4 = ax[1,1].pcolormesh(u.cpu(), v.cpu(), psf[nn,1,1].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh5 = ax[0,2].pcolormesh(u.cpu(), v.cpu(), psf[nn,2,0].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh6 = ax[1,2].pcolormesh(u.cpu(), v.cpu(), psf[nn,2,1].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
ax[0,0].set_xlim((-150,150))
ax[0,0].set_ylim((-150,150))
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')
ax[0,1].set_xlim((-150,150))
ax[0,1].set_ylim((-150,150))
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')
ax[1,0].set_xlim((-150,150))
ax[1,0].set_ylim((-150,150))
cb = plt.colorbar(mesh3, pad=0.15, label='Photon number')
ax[1,1].set_xlim((-150,150))
ax[1,1].set_ylim((-150,150))
cb = plt.colorbar(mesh4, pad=0.15, label='Photon number')
ax[0,2].set_xlim((-150,150))
ax[0,2].set_ylim((-150,150))
cb = plt.colorbar(mesh5, pad=0.15, label='Photon number')
ax[1,2].set_xlim((-150,150))
ax[1,2].set_ylim((-150,150))
cb = plt.colorbar(mesh6, pad=0.15, label='Photon number')

In [ ]:
def signal(psf):
    return torch.mean(torch.max(torch.flatten(psf, start_dim=1), dim=1)[0])

In [ ]:
def general_loss(M, rho, eta, delta, N_photons, data):
    h = PSF(rho=rho, eta=eta, delta=delta, M=M, N_photons=N_photons)
    #
    # loss = torch.sum(torch.add(h, -data).pow(2))
    loss = torch.sum(torch.add(h, -(data+sig_r**2)*torch.log(h+B+sig_r**2)))
    #rho_bound = limit(rho, 180, 100, upper=True) + limit(rho, 0, 100, upper=False)
    delta_bound = limit(delta, 180, 100, upper=True) + limit(delta, 1, 100, upper=False)
    #eta_bound = limit(eta, 180, 100, upper=True) + limit(eta, 0, 100, upper=False)
    return loss + 1000.*(delta_bound) #+ 0.1*torch.sum(h.pow(2))

In [ ]:
background = 11.
sig_b = 6.
read=1.5
bias=11.
psf_n = noise(psf, QE=1, EM=1, b=background, sigma_b=sig_b, sigma_r=read, bias=bias)

In [ ]:
print('z = ', z[nn].cpu().detach().numpy())

vmax=torch.max(psf_n[nn]).cpu().detach().numpy()
vmin=torch.min(psf_n[nn]).cpu().detach().numpy()

plt.rcParams['figure.figsize'] = [15,9]
fig, ax = plt.subplots(2,3)
mesh1 = ax[0,0].pcolormesh(u.cpu(), v.cpu(), psf_n[nn,0,0].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh2 = ax[1,0].pcolormesh(u.cpu(), v.cpu(), psf_n[nn,0,1].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh3 = ax[0,1].pcolormesh(u.cpu(), v.cpu(), psf_n[nn,1,0].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh4 = ax[1,1].pcolormesh(u.cpu(), v.cpu(), psf_n[nn,1,1].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh5 = ax[0,2].pcolormesh(u.cpu(), v.cpu(), psf_n[nn,2,0].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
mesh6 = ax[1,2].pcolormesh(u.cpu(), v.cpu(), psf_n[nn,2,1].cpu().detach().numpy(), cmap='gray', vmin=vmin, vmax=vmax)
ax[0,0].set_xlim((-150,150))
ax[0,0].set_ylim((-150,150))
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')
ax[0,1].set_xlim((-150,150))
ax[0,1].set_ylim((-150,150))
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')
ax[1,0].set_xlim((-150,150))
ax[1,0].set_ylim((-150,150))
cb = plt.colorbar(mesh3, pad=0.15, label='Photon number')
ax[1,1].set_xlim((-150,150))
ax[1,1].set_ylim((-150,150))
cb = plt.colorbar(mesh4, pad=0.15, label='Photon number')
ax[0,2].set_xlim((-150,150))
ax[0,2].set_ylim((-150,150))
cb = plt.colorbar(mesh5, pad=0.15, label='Photon number')
ax[1,2].set_xlim((-150,150))
ax[1,2].set_ylim((-150,150))
cb = plt.colorbar(mesh6, pad=0.15, label='Photon number')

In [ ]:
B = background + bias
sig_r = np.sqrt(read**2 + sig_b**2)

params = torch.tensor(np.concatenate((rho.cpu().clone().detach()+40.*(np.random.rand(300)-0.5), #rho
                                     np.array([90. for i in range(300)]), #eta
                                     delta.cpu().clone().detach()+40.*(np.random.rand(300)-0.5))),
                                     requires_grad=True, device=device)
params_prev = params.clone().detach() - torch.tensor([2. for k in range(len(params))], device=device)
'''
thresh_rho = 10.
thresh_delta = 10.
rho_guess = params[0:100].clone().detach()
delta_guess = params[200:300].clone().detach()

lambd = 1.
'''

# Use Stochastic Gradient Descent (SGD) to optimize params
optimizer = torch.optim.Adam([params], lr=0.9)  # Learning rate = 0.01

num_epochs_max = 300
loss_ = []
rho_ = []
eta_ = []
delta_ = []
N_p = []

tour=0

#for u in range(5000):
while (torch.add(params, -params_prev)>.001).sum()>3 and tour<num_epochs_max:
    params_prev = params.clone().detach()
    optimizer.zero_grad()  # Reset gradients
    #print(params[0:3], params[3:6], params[6:])
    loss = general_loss(M.detach(), params[0:300], params[300:600], params[600:], N_photons.detach(), psf_n.detach())
    #loss = general_loss(M.detach(), params[0], params[1], params[2], N_photons.detach(), psf.detach())
    loss_.append(loss.cpu().detach().numpy())

    rho___ = params[0:300].cpu().clone().detach().numpy()%180
    rho___[rho___-rho.cpu().clone().detach().numpy() > 90] = rho___[rho___-rho.cpu().clone().detach().numpy() > 90] - 180.
    rho___[rho___-rho.cpu().clone().detach().numpy() < -90] = rho___[rho___-rho.cpu().clone().detach().numpy() < -90] + 180.
    eta___ = params[300:600].cpu().clone().detach().numpy()%180
    eta___[eta___-eta.cpu().clone().detach().numpy() > 90] = eta___[eta___-eta.cpu().clone().detach().numpy() > 90] - 180.
    eta___[eta___-eta.cpu().clone().detach().numpy() < -90] = eta___[eta___-eta.cpu().clone().detach().numpy() < -90] + 180.
    rho_.append(rho___)
    eta_.append(eta___)
    delta_.append(params[600:].cpu().clone().detach().numpy())

    loss.backward()  # Backpropagation
    optimizer.step()  # Update parameters
    tour+=1

In [ ]:
signal_ = signal(psf.cpu().detach())
print(signal_/sig_r)

In [ ]:
plt.rcParams['figure.figsize'] = [17,2.5]
plt.rcParams.update({'font.size': 9})
fig, ax = plt.subplots(1,4)
ax[0].plot(loss_)    
ax[0].set_xlabel('Iteration')
ax[0].set_ylabel('Loss')
#ax[0].set_ylim((np.min(loss_), np.max(loss_)))
ax[0].grid()
ax[1].plot(np.array(rho_)-rho.cpu().clone().detach().numpy())
ax[1].set_xlabel('Iteration')
ax[1].set_ylabel('$\\Delta \\rho$ ($\\degree$)')
ax[1].set_title('std = '+str(int(1000*np.nanstd((np.array(rho_)-rho.cpu().clone().detach().numpy())[:][-1]))/1000)+'\n mean = '+str(int(1000*np.nanmean((np.array(rho_)-rho.cpu().clone().detach().numpy())[:][-1]))/1000))
ax[1].grid()
ax[2].plot(eta_)
ax[2].axhline(eta[0].cpu().detach().numpy(), c='r', label='Ground truth')
ax[2].legend()
ax[2].set_xlabel('Iteration')
ax[2].set_ylabel('$\\eta$ ($\\degree$)')
ax[2].set_title('std = '+str(int(1000*np.nanstd(eta_[:][-1]))/1000)+'\n mean = '+str(int(1000*np.nanmean(eta_[:][-1]))/1000))
ax[2].grid()
ax[3].plot(delta_)
ax[3].axhline(delta[0].cpu().detach().numpy(), c='r', label='Ground truth')
ax[3].legend()
ax[3].set_xlabel('Iteration')
ax[3].set_ylabel('$\\delta$ ($\\degree$)')
ax[3].set_title('std = '+str(int(1000*np.nanstd(delta_[:][-1]))/1000)+'\n mean = '+str(int(1000*np.nanmean(delta_[:][-1]))/1000))
ax[3].grid()

In [ ]:
N_photons

In [ ]:
N_photons

In [ ]:
np.savez_compressed('D:/AMAURY/data_simu/stage/z_study.npz', signal=signal_.detach().numpy(), std=sig_r, floor=B, rho=np.array(rho_-rho.cpu().clone().detach().numpy())[-1,:], eta=np.array(eta_)[-1,:], delta=np.array(delta_)[-1,:], z=z.cpu().detach().numpy())

In [ ]:
data = np.load('D:/AMAURY/data_simu/stage/z_study.npz')
rhos = data['rho']
etas=data['eta']
deltas=data['delta']
stds=data['std']
signals=data['signal']
floors=data['floor']
z = data['z']

In [ ]:
plt.rcParams['figure.figsize'] = [17,3.5]
plt.rcParams.update({'font.size': 11})
fig, ax = plt.subplots(1,3)

ax[0].scatter(z, rhos, c='r', s=5)
ax[0].grid()
ax[0].axvline(1.5, label='Focal planes')
ax[0].axvline(1.5-0.35)
ax[0].axvline(1.5+0.35)
ax[0].set_ylabel('$\\Delta \\rho$ ($^{\\circ}$)')
ax[0].set_xlabel('z ($\mu$m)')
ax[0].legend()
#ax[0].set_ylim((-90,90))

ax[1].scatter(z, etas, c='b', s=5, label='Eta')
ax[1].axhline(45., c='r', label='ground truth')
ax[1].axvline(1.5, label='Focal planes')
ax[1].axvline(1.5-0.35)
ax[1].axvline(1.5+0.35)
ax[1].grid()
ax[1].set_ylabel('$\\eta$ ($^{\\circ}$)')
ax[1].set_xlabel('z ($\mu$m)')
ax[1].legend()
ax[1].set_ylim((0,180))

ax[2].scatter(z, deltas, c='g', s=5, label='Delta')
ax[2].axhline(80., c='r', label='ground truth')
ax[2].axvline(1.5, label='Focal planes')
ax[2].axvline(1.5-0.35)
ax[2].axvline(1.5+0.35)
ax[2].grid()
ax[2].set_ylabel('$\\delta$ ($^{\\circ}$)')
ax[2].set_xlabel('z ($\mu$m)')
ax[2].legend()
ax[2].set_ylim((0,180))